In [1]:
pip install sarvamai

In [22]:
pip install firebase_admin

   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.5/4.3 MB 2.4 MB/s eta 0:00:02
   ------------ --------------------------- 1.3/4.3 MB 3.4 MB/s eta 0:00:01
   --------------------- ------------------ 2.4/4.3 MB 3.9 MB/s eta 0:00:01
   ----------------------------- ---------- 3.1/4.3 MB 3.8 MB/s eta 0:00:01
   -------------------------------------- - 4.2/4.3 MB 3.9 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 3.8 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.8.0
    Uninstalling PyJWT-2.8.0:
      Successfully uninstalled PyJWT-2.8.0
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.70.0
    Uninstalling grpcio-1.70.0:
      Successfully uninstalled grpcio-1.70.0
  Attempt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aext-assistant-server 4.1.0 requires anaconda-cloud-auth>=0.7.1, which is not installed.
aext-shared 4.1.0 requires anaconda-cloud-auth>=0.7.1, which is not installed.
aext-share-notebook-server 4.1.0 requires anaconda-cloud-auth>=0.7.1, which is not installed.
aext-toolbox 4.1.0 requires anaconda-cloud-auth>=0.7.1, which is not installed.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.31.1 which is incompatible.
tensorflow-intel 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatible.


In [5]:
from sarvamai import SarvamAI

client = SarvamAI(
    api_subscription_key="sk_ne661iqu_FmeMG9IRLcchGkwM0EDSzZvz",
)

response = client.text.translate(
    input="Hi, My Name is Vinayak.",
    source_language_code="auto",
    target_language_code="gu-IN",
    speaker_gender="Male"
)




In [24]:
from flask import Flask,request,jsonify,render_template
from flask_cors import CORS
import firebase_admin
from firebase_admin import credentials,auth,firestore

In [26]:
# app.py
from flask import Flask, request, jsonify, render_template
from flask_cors import CORS
import firebase_admin
from firebase_admin import credentials, auth, firestore
import os
import requests
import base64
import json

# --- Flask App Setup ---
app = Flask(__name__)
CORS(app) # Enable CORS for frontend communication

# --- Firebase Admin SDK Initialization ---
# IMPORTANT: Replace with your Firebase Admin SDK private key file path
# For security, do NOT hardcode this in production. Use environment variables or a secure key management system.
# You can download this JSON file from Firebase Project Settings -> Service accounts
# For local development, place it in the same directory as app.py
FIREBASE_ADMIN_SDK_PATH = os.getenv('FIREBASE_ADMIN_SDK_PATH', 'path/to/your/serviceAccountKey.json')

try:
    cred = credentials.Certificate(FIREBASE_ADMIN_SDK_PATH)
    firebase_admin.initialize_app(cred)
    db = firestore.client()
    print("Firebase Admin SDK initialized successfully.")
except Exception as e:
    print(f"Error initializing Firebase Admin SDK: {e}")
    print("Please ensure 'path/to/your/serviceAccountKey.json' is correct and accessible.")
    db = None # Set db to None if initialization fails

# --- API Keys and Endpoints ---
# Gemini API Key (Canvas will inject this for frontend, but backend needs it for direct calls)
# For production, use environment variables: os.getenv('GEMINI_API_KEY')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY', '') # Replace with your Gemini API Key if running outside Canvas

# SarvamAI TTS Configuration
SARVAM_API_KEY = os.getenv('SARVAM_API_KEY', 'sk_ne661iqu_FmeMG9IRLchGkwM0EDSZzZvz') # Your SarvamAI API Key
# !!! IMPORTANT: Replace with the ACTUAL SarvamAI Text-to-Speech REST API endpoint.
# !!! This is the URL that caused the 404 error.
SARVAM_TTS_API_URL = os.getenv('SARVAM_TTS_API_URL', 'https://api.sarvam.ai/v1/text-to-speech/convert') # Placeholder - CORRECT THIS!

# !!! IMPORTANT: Replace with the ACTUAL SarvamAI Speech-to-Text REST API endpoint.
# !!! This is a hypothetical URL based on common API patterns.
SARVAM_STT_API_URL = os.getenv('SARVAM_STT_API_URL', 'https://api.sarvam.ai/v1/speech-to-text/transcribe') # Placeholder - CORRECT THIS!

GEMINI_FLASH_API_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"

# Global app ID for Firestore paths (should match frontend)
APP_ID = "default-debate-app" # This should match __app_id in the Canvas environment

# --- Helper Functions for AI Interactions ---

def call_gemini_api(prompt_text):
    """Calls the Gemini API to generate text."""
    headers = {'Content-Type': 'application/json'}
    params = {'key': GEMINI_API_KEY}
    payload = {
        "contents": [{"role": "user", "parts": [{"text": prompt_text}]}]
    }
    try:
        response = requests.post(GEMINI_FLASH_API_URL, headers=headers, params=params, json=payload)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        result = response.json()
        if result.get('candidates') and len(result['candidates']) > 0 and \
           result['candidates'][0].get('content') and result['candidates'][0]['content'].get('parts') and \
           len(result['candidates'][0]['content']['parts']) > 0:
            return result['candidates'][0]['content'].get('parts')[0].get('text', "I'm sorry, I couldn't generate a response.")
        else:
            print(f"Unexpected Gemini API response structure: {result}")
            return "I'm sorry, I couldn't generate a response."
    except requests.exceptions.RequestException as e:
        print(f"Error calling Gemini API: {e}")
        return f"Error: Could not get AI response. Please try again. ({e})"

def call_sarvam_tts_api(text_to_speak):
    """Calls the SarvamAI TTS API to convert text to speech."""
    headers = {
        'Content-Type': 'application/json',
        'api-subscription-key': SARVAM_API_KEY,
    }
    payload = {
        "text": text_to_speak,
        "target_language_code": "en-IN",
        "speaker": "vidya", # Assuming 'vidya' speaker is still valid
        "pitch": 0,
        "pace": 1,
        "loudness": 1,
        "speech_sample_rate": 22050,
        "enable_preprocessing": True,
        "model": "saarika:v2.5" # Updated model as per user's input
    }
    try:
        response = requests.post(SARVAM_TTS_API_URL, headers=headers, json=payload)
        response.raise_for_status()
        # Assuming SarvamAI returns audio data directly or as a base64 string
        # If it returns a JSON with a base64 field, you'll need to parse that.
        return base64.b64encode(response.content).decode('utf-8') # Return base64 encoded audio
    except requests.exceptions.RequestException as e:
        print(f"Error calling SarvamAI TTS API: {e}")
        print(f"SarvamAI TTS Response content: {response.content.decode('utf-8', errors='ignore') if response.content else 'No content'}")
        return None # Indicate failure

def call_sarvam_stt_api(audio_base64_data):
    """Calls the SarvamAI STT API to transcribe audio."""
    headers = {
        'Content-Type': 'application/json', # Or 'audio/wav' if sending raw bytes
        'api-subscription-key': SARVAM_API_KEY,
    }
    # Assuming the STT API expects base64 encoded audio in a JSON payload.
    # Adjust 'audio_base64' key if SarvamAI uses a different one.
    payload = {
        "audio_base64": audio_base64_data,
        "language_code": "en-IN" # Specify language for transcription
    }
    try:
        response = requests.post(SARVAM_STT_API_URL, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        transcript = result.get('transcript')
        if transcript:
            return transcript
        else:
            print(f"Unexpected SarvamAI STT response structure: {result}")
            return ""
    except requests.exceptions.RequestException as e:
        print(f"Error calling SarvamAI STT API: {e}")
        print(f"SarvamAI STT Response content: {response.content.decode('utf-8', errors='ignore') if response.content else 'No content'}")
        return ""

# --- Debate Turn Configuration (Python equivalent of React's getDebateTurns) ---
def get_debate_turns_config(chosen_side, ai_skill_level):
    pro_prompt = lambda topic: f"You are the user. The debate topic is \"{topic}\". Please make your opening statement for the PRO side. Be concise and persuasive."
    con_prompt = lambda topic: f"You are the user. The debate topic is \"{topic}\". Please make your opening statement for the CON side. Be concise and persuasive."

    ai_pro_rebuttal = lambda topic, last_user_statement: f"You are an AI debater. The debate topic is \"{topic}\". Your opponent (the user) just made their opening statement: \"{last_user_statement}\". Please provide a concise rebuttal against their points, arguing for the PRO side. Your skill level is \"{ai_skill_level}\"."
    ai_con_rebuttal = lambda topic, last_user_statement: f"You are an AI debater. The debate topic is \"{topic}\". Your opponent (the user) just made their opening statement: \"{last_user_statement}\". Please provide a concise rebuttal against their points, arguing for the CON side. Your skill level is \"{ai_skill_level}\"."

    ai_pro_closing = lambda topic, last_user_statement: f"You are an AI debater. The debate topic is \"{topic}\". Your opponent (the user) just made their rebuttal: \"{last_user_statement}\". Please provide a concise closing statement for the PRO side, summarizing your main points and refuting the opponent's. Your skill level is \"{ai_skill_level}\"."
    ai_con_closing = lambda topic, last_user_statement: f"You are an AI debater. The debate topic is \"{topic}\". Your opponent (the user) just made their rebuttal: \"{last_user_statement}\". Please provide a concise closing statement for the CON side, summarizing your main points and refuting the opponent's. Your skill level is \"{ai_skill_level}\"."

    if chosen_side == 'pro':
        return [
            {'speaker': 'user', 'type': 'opening', 'prompt': pro_prompt},
            {'speaker': 'ai', 'type': 'rebuttal', 'prompt': ai_con_rebuttal},
            {'speaker': 'user', 'type': 'rebuttal', 'prompt': lambda topic, last_ai_statement: f"You are the user. The debate topic is \"{topic}\". The AI just made a rebuttal: \"{last_ai_statement}\". Please provide your rebuttal to the AI's points, reinforcing your PRO side."},
            {'speaker': 'ai', 'type': 'closing', 'prompt': ai_con_closing},
            {'speaker': 'user', 'type': 'closing', 'prompt': lambda topic, last_ai_statement: f"You are the user. The debate topic is \"{topic}\". The AI just made its closing statement: \"{last_ai_statement}\". Please provide your closing statement for the PRO side."},
        ]
    else: # chosen_side == 'con'
        return [
            {'speaker': 'user', 'type': 'opening', 'prompt': con_prompt},
            {'speaker': 'ai', 'type': 'rebuttal', 'prompt': ai_pro_rebuttal},
            {'speaker': 'user', 'type': 'rebuttal', 'prompt': lambda topic, last_ai_statement: f"You are the user. The debate topic is \"{topic}\". The AI just made a rebuttal: \"{last_ai_statement}\". Please provide your rebuttal to the AI's points, reinforcing your CON side."},
            {'speaker': 'ai', 'type': 'closing', 'prompt': ai_pro_closing},
            {'speaker': 'user', 'type': 'closing', 'prompt': lambda topic, last_ai_statement: f"You are the user. The debate topic is \"{topic}\". The AI just made its closing statement: \"{last_ai_statement}\". Please provide your closing statement for the CON side."},
        ]

# --- Flask Routes ---

@app.route('/')
def index():
    """Serves the main HTML page."""
    return render_template('index.html')

@app.route('/api/auth/verify_token', methods=['POST'])
def verify_token():
    """Verifies Firebase ID token and returns user info."""
    if not firebase_admin._apps:
        return jsonify({"error": "Firebase not initialized"}), 500

    id_token = request.json.get('idToken')
    if not id_token:
        return jsonify({"error": "No ID token provided"}), 400

    try:
        decoded_token = auth.verify_id_token(id_token)
        uid = decoded_token['uid']
        user_record = auth.get_user(uid)
        return jsonify({
            "uid": uid,
            "email": user_record.email,
            "displayName": user_record.display_name,
            "isAnonymous": user_record.anonymous
        }), 200
    except Exception as e:
        print(f"Token verification failed: {e}")
        return jsonify({"error": str(e)}), 401

@app.route('/api/speech_to_text', methods=['POST'])
def speech_to_text():
    """Handles speech-to-text transcription using SarvamAI."""
    data = request.json
    audio_base64 = data.get('audioBase64')
    if not audio_base64:
        return jsonify({"error": "No audio data provided"}), 400

    transcript = call_sarvam_stt_api(audio_base64)
    if transcript:
        return jsonify({"transcript": transcript})
    else:
        return jsonify({"error": "Failed to transcribe audio"}), 500


@app.route('/api/debate/ai_turn', methods=['POST'])
def ai_turn():
    """Handles AI's debate turn, generates response and speech."""
    data = request.json
    topic = data.get('topic')
    last_opponent_statement = data.get('lastOpponentStatement')
    current_turn_index = data.get('currentTurnIndex')
    ai_skill_level = data.get('aiSkillLevel')
    user_side = data.get('userSide')
    debate_log = data.get('debateLog', [])

    if not all([topic, last_opponent_statement is not None, current_turn_index is not None, ai_skill_level, user_side]):
        return jsonify({"error": "Missing required parameters"}), 400

    debate_turns_config = get_debate_turns_config(user_side, ai_skill_level)

    if current_turn_index >= len(debate_turns_config):
        return jsonify({"error": "Debate turns exhausted"}), 400

    current_turn_data = debate_turns_config[current_turn_index]
    
    # Filter out system messages from previous AI arguments
    previous_ai_arguments = "\n".join([
        log['text'] for log in debate_log if log['speaker'] == 'ai' and log['type'] != 'system'
    ])

    prompt = current_turn_data['prompt'](topic, last_opponent_statement, previous_ai_arguments)
    ai_text = call_gemini_api(prompt)

    audio_base64 = None
    if ai_text and ai_text != "I'm sorry, I couldn't generate a response.":
        audio_base64 = call_sarvam_tts_api(ai_text)

    return jsonify({
        "aiText": ai_text,
        "audioBase64": audio_base64
    })

@app.route('/api/debate/feedback', methods=['POST'])
def judge_feedback():
    """Generates judge's feedback based on the WUDC manual."""
    data = request.json
    topic = data.get('topic')
    user_case = data.get('userCase')
    debate_log = data.get('debateLog')
    user_side = data.get('userSide')

    if not all([topic, user_case is not None, debate_log, user_side]):
        return jsonify({"error": "Missing required parameters"}), 400

    full_debate_transcript = "\n".join([
        f"{entry['speaker'].capitalize()}: {entry['text']}" for entry in debate_log
    ])

    prompt = f"""You are a debate judge, evaluating a British Parliamentary debate based on the WUDC Debating & Judging Manual (Panama WUDC 2025).
    The debate topic was "{topic}". The user's initial case preparation was: "{user_case}".
    The user argued the {user_side.upper()} side.
    Here is the full debate transcript:
    {full_debate_transcript}

    Please provide constructive feedback on the user's performance, focusing on the following WUDC criteria:
    1.  **Persuasiveness:** How well did the user's arguments convince an "ordinary intelligent voter"?
    2.  **Analysis (Matter):** Evaluate the reasons offered for their arguments. Were they precise, detailed, and plausible? Did they identify logical implications or emotional significance?
    3.  **Style (Manner):** Assess clarity, comprehensibility, and effective conveyance of meaning and significance. Avoid judging accent or subjective aesthetic preferences. Was the user's speech easy to follow and understand? Did their delivery amplify their logical analysis?
    4.  **Rebuttal, Engagement, and Comparisons:** How well did the user engage with the AI's arguments? Did they effectively respond to points, or were significant AI claims left unchallenged?
    5.  **Role Fulfilment:** Did the user fulfill their role (e.g., making opening statement, providing rebuttals, closing statement)? (Note: POI engagement is not explicitly tracked here, but general engagement is.)
    6.  **Case Preparation Quality:** Based on their initial case preparation and how it translated into their arguments.

    Provide specific examples from the transcript to support your feedback. Conclude with a general assessment of the AI's speech quality and interactivity based on the flow. Structure your feedback clearly and concisely, aiming for a tone that is helpful and evaluative, similar to a WUDC judge's oral adjudication."""

    feedback_text = call_gemini_api(prompt)

    audio_base64 = None
    if feedback_text and feedback_text != "I'm sorry, I couldn't generate a response.":
        audio_base64 = call_sarvam_tts_api(feedback_text)

    return jsonify({
        "judgeFeedback": feedback_text,
        "audioBase64": audio_base64
    })

@app.route('/api/debates/save', methods=['POST'])
def save_debate():
    """Saves a debate to Firestore."""
    if not db:
        return jsonify({"error": "Firestore not initialized"}), 500

    data = request.json
    user_id = data.get('userId')
    debate_data = data.get('debateData')

    if not all([user_id, debate_data]):
        return jsonify({"error": "Missing user ID or debate data"}), 400

    try:
        debate_data['timestamp'] = firestore.SERVER_TIMESTAMP
        doc_ref = db.collection(f'artifacts/{APP_ID}/users/{user_id}/debates').add(debate_data)
        return jsonify({"message": "Debate saved successfully", "id": doc_ref[1].id}), 201
    except Exception as e:
        print(f"Error saving debate: {e}")
        return jsonify({"error": str(e)}), 500

@app.route('/api/debates/load/<user_id>', methods=['GET'])
def load_debates(user_id):
    """Loads all debates for a user from Firestore."""
    if not db:
        return jsonify({"error": "Firestore not initialized"}), 500

    try:
        debates_ref = db.collection(f'artifacts/{APP_ID}/users/{user_id}/debates').order_by('timestamp', direction=firestore.Query.DESCENDING).stream()
        debates = []
        for doc in debates_ref:
            data = doc.to_dict()
            if 'timestamp' in data and data['timestamp']:
                data['timestamp'] = data['timestamp'].isoformat()
            debates.append({"id": doc.id, **data})
        return jsonify(debates), 200
    except Exception as e:
        print(f"Error loading debates: {e}")
        return jsonify({"error": str(e)}), 500

# --- Running the Flask App ---
if __name__ == '__main__':
    if not os.path.exists('templates'):
        os.makedirs('templates')
    app.run(debug=True)

Error initializing Firebase Admin SDK: [Errno 2] No such file or directory: 'path/to/your/serviceAccountKey.json'
Please ensure 'path/to/your/serviceAccountKey.json' is correct and accessible.
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\DELL\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [28]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>IDL Debate Simulator</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap" rel="stylesheet">
    <!-- Lucide Icons (for React, but we can use them via CDN for simple HTML) -->
    <script src="https://unpkg.com/lucide@latest/dist/umd/lucide.js"></script>
    <style>
        body {
            font-family: 'Inter', sans-serif;
        }
        .debate-log-entry {
            display: flex;
            align-items: flex-start;
            gap: 0.75rem; /* gap-3 */
            padding: 0.5rem; /* p-2 */
            border-radius: 0.5rem; /* rounded-lg */
            box-shadow: 0 1px 2px 0 rgba(0, 0, 0, 0.05); /* shadow-sm */
            max-width: 85%;
        }
        .debate-log-entry.user {
            background-color: rgba(239, 246, 255, 0.7); /* bg-blue-50/70 */
            align-self: flex-end;
            margin-left: auto; /* Push to right */
        }
        .debate-log-entry.ai {
            background-color: rgba(249, 250, 251, 0.7); /* bg-gray-50/70 */
            align-self: flex-start;
            margin-right: auto; /* Push to left */
        }
        .debate-log-entry.system {
            background-color: rgba(254, 252, 232, 0.7); /* bg-yellow-50/70 */
            color: #4b5563; /* text-gray-700 */
            align-self: center;
            margin-left: auto;
            margin-right: auto;
        }
        .icon-svg {
            width: 1.5rem; /* w-6 */
            height: 1.5rem; /* h-6 */
            flex-shrink: 0;
        }
        .icon-svg.user { color: #2563eb; /* text-blue-600 */ }
        .icon-svg.bot { color: #4b5563; /* text-gray-600 */ }
        .icon-svg.message-square { color: #d97706; /* text-yellow-600 */ }
        .icon-svg.award { color: #1d4ed8; /* text-blue-800 */ }
        .icon-svg.book-open { color: #16a34a; /* text-green-600 */ }
        .icon-svg.mic { color: #9333ea; /* text-purple-600 */ }
        .icon-svg.loader { color: #3b82f6; /* text-blue-500 */ animation: spin 1s linear infinite; }
        @keyframes spin {
            from { transform: rotate(0deg); }
            to { transform: rotate(360deg); }
        }
        .animate-pulse { animation: pulse 2s cubic-bezier(0.4, 0, 0.6, 1) infinite; }
        @keyframes pulse {
            0%, 100% { opacity: 1; }
            50% { opacity: .5; }
        }
    </style>
    <!-- Firebase SDKs (client-side) -->
    <script type="module">
        import { initializeApp } from "https://www.gstatic.com/firebasejs/10.12.2/firebase-app.js";
        import { getAuth, signInWithPopup, GoogleAuthProvider, signOut, onAuthStateChanged, signInAnonymously, signInWithCustomToken, createUserWithEmailAndPassword, signInWithEmailAndPassword, sendPasswordResetEmail } from "https://www.gstatic.com/firebasejs/10.12.2/firebase-auth.js";
        import { getFirestore, doc, getDoc, addDoc, setDoc, updateDoc, deleteDoc, onSnapshot, collection, query, orderBy, getDocs, serverTimestamp } from "https://www.gstatic.com/firebasejs/10.12.2/firebase-firestore.js";

        // IMPORTANT: These are global variables provided by the Canvas environment.
        // For local testing, you might need to hardcode them or load from a config.
        const __app_id = window.__app_id || 'default-debate-app';
        const __firebase_config = window.__firebase_config || JSON.stringify({});
        const __initial_auth_token = window.__initial_auth_token || null;

        // Initialize Firebase
        const firebaseConfig = JSON.parse(__firebase_config);
        window.firebaseApp = initializeApp(firebaseConfig);
        window.firebaseAuth = getAuth(window.firebaseApp);
        window.firebaseDb = getFirestore(window.firebaseApp);

        // Make Firebase functions globally accessible (for simplicity in this HTML file)
        window.auth = window.firebaseAuth;
        window.db = window.firebaseDb;
        window.APP_ID = __app_id;
        window.initialAuthToken = __initial_auth_token;
        window.GoogleAuthProvider = GoogleAuthProvider;
        window.onAuthStateChanged = onAuthStateChanged;
        window.signInAnonymously = signInAnonymously;
        window.signInWithCustomToken = signInWithCustomToken;
        window.signInWithEmailAndPassword = signInWithEmailAndPassword;
        window.createUserWithEmailAndPassword = createUserWithEmailAndPassword;
        window.sendPasswordResetEmail = sendPasswordResetEmail;
        window.signOut = signOut;
        window.collection = collection;
        window.addDoc = addDoc;
        window.onSnapshot = onSnapshot;
        window.query = query;
        window.orderBy = orderBy;
        window.serverTimestamp = serverTimestamp;
        window.signInWithPopup = signInWithPopup;
    </script>
</head>
<body class="min-h-screen bg-gradient-to-br from-blue-50 to-indigo-100 text-gray-900 p-4 sm:p-6 flex items-center justify-center">
    <div id="app" class="w-full max-w-6xl bg-white rounded-xl shadow-2xl overflow-hidden flex flex-col md:flex-row">
        <!-- Left Panel: Controls and Setup -->
        <div class="md:w-1/3 p-6 bg-blue-50 border-b md:border-b-0 md:border-r border-blue-200 flex flex-col gap-6">
            <h1 class="text-2xl font-bold text-blue-800 flex items-center gap-2">
                <svg class="icon-svg award"><use href="#award"></use></svg> Debate Simulator
            </h1>
            <p class="text-sm text-gray-700">Practice your debating skills against an AI opponent.</p>

            <!-- User Authentication -->
            <div class="flex flex-col gap-3 bg-blue-100 p-4 rounded-md shadow-sm">
                <div class="flex items-center justify-between text-sm text-gray-700">
                    <div id="user-display" class="flex items-center gap-2">
                        <svg class="icon-svg user"><use href="#user"></use></svg>
                        <span>Loading user...</span>
                    </div>
                    <button id="auth-button" class="bg-blue-500 text-white py-1.5 px-3 rounded-md hover:bg-blue-600 transition-colors duration-200 flex items-center gap-1">
                        <svg class="icon-svg"><use href="#log-in"></use></svg> Login
                    </button>
                </div>
                <p id="full-user-id-display" class="text-xs text-gray-500 mt-1 hidden">Your full user ID: <span class="font-mono break-all" id="user-id-value"></span></p>

                <div id="auth-forms" class="flex flex-col gap-2 mt-2 hidden">
                    <input type="email" id="email-input" placeholder="Email" class="p-2 border border-blue-300 rounded-md text-sm" />
                    <input type="password" id="password-input" placeholder="Password" class="p-2 border border-blue-300 rounded-md text-sm" />
                    <div class="flex gap-2">
                        <button id="email-login-button" class="flex-1 bg-blue-600 text-white py-1.5 px-3 rounded-md hover:bg-blue-700 transition-colors duration-200 flex items-center justify-center gap-1">
                            <svg class="icon-svg"><use href="#log-in"></use></svg> Login
                        </button>
                        <button id="email-register-button" class="flex-1 bg-green-600 text-white py-1.5 px-3 rounded-md hover:bg-green-700 transition-colors duration-200 flex items-center justify-center gap-1">
                            <svg class="icon-svg"><use href="#user-plus"></use></svg> Register
                        </button>
                    </div>
                    <button id="password-reset-button" class="text-blue-600 text-xs hover:underline mt-1 self-start">
                        Forgot Password?
                    </button>
                    <p id="auth-error-display" class="text-red-500 text-xs mt-1 hidden"></p>
                </div>
            </div>

            <!-- Debate Topic Input -->
            <div class="flex flex-col gap-2">
                <label for="topic" class="text-sm font-medium text-gray-700 flex items-center gap-1">
                    <svg class="icon-svg"><use href="#book-open"></use></svg> Debate Topic
                </label>
                <input id="topic" type="text" placeholder="e.g., 'AI will replace human jobs'" class="p-2 border border-blue-300 rounded-md focus:ring-2 focus:ring-blue-400 focus:border-transparent outline-none text-sm" />
            </div>

            <!-- Choose Your Side -->
            <div class="flex flex-col gap-2">
                <label class="text-sm font-medium text-gray-700 flex items-center gap-1">
                    <svg class="icon-svg"><use href="#user"></use></svg> Choose Your Side
                </label>
                <div class="flex gap-4">
                    <label class="inline-flex items-center">
                        <input type="radio" class="form-radio text-blue-600" name="userSide" value="pro" checked />
                        <span class="ml-2 text-sm text-gray-700">PRO</span>
                    </label>
                    <label class="inline-flex items-center">
                        <input type="radio" class="form-radio text-blue-600" name="userSide" value="con" />
                        <span class="ml-2 text-sm text-gray-700">CON</span>
                    </label>
                </div>
            </div>

            <!-- AI Skill Level Selector -->
            <div class="flex flex-col gap-2">
                <label for="skill-level" class="text-sm font-medium text-gray-700 flex items-center gap-1">
                    <svg class="icon-svg"><use href="#bot"></use></svg> AI Skill Level
                </label>
                <select id="skill-level" class="p-2 border border-blue-300 rounded-md focus:ring-2 focus:ring-blue-400 focus:border-transparent outline-none text-sm bg-white">
                    <option value="beginner">Beginner</option>
                    <option value="intermediate">Intermediate</option>
                    <option value="advanced">Advanced</option>
                </select>
            </div>

            <!-- Case Preparation -->
            <div class="flex flex-col gap-2">
                <label for="user-case" class="text-sm font-medium text-gray-700 flex items-center gap-1">
                    <svg class="icon-svg"><use href="#book-open"></use></svg> Your Case Preparation
                </label>
                <textarea id="user-case" placeholder="Outline your main arguments for your chosen side here. This helps the judge evaluate your preparation." rows="5" class="p-2 border border-blue-300 rounded-md focus:ring-2 focus:ring-blue-400 focus:border-transparent outline-none text-sm resize-y"></textarea>
            </div>

            <!-- Action Buttons -->
            <div class="flex flex-col gap-3 mt-auto">
                <button id="start-debate-button" class="w-full bg-blue-600 text-white py-2 px-4 rounded-lg hover:bg-blue-700 transition-colors duration-200 flex items-center justify-center gap-2 shadow-md">
                    <svg class="icon-svg"><use href="#message-square"></use></svg> Start Debate
                </button>
                <button id="reset-debate-button" class="w-full bg-red-500 text-white py-2 px-4 rounded-lg hover:bg-red-600 transition-colors duration-200 flex items-center justify-center gap-2 shadow-md hidden">
                    <svg class="icon-svg"><use href="#refresh-cw"></use></svg> Reset Debate
                </button>
            </div>

            <!-- Past Debates Section -->
            <div id="past-debates-section" class="mt-6 hidden">
                <h3 class="text-lg font-semibold text-blue-800 mb-3">Your Past Debates</h3>
                <div id="past-debates-list" class="max-h-60 overflow-y-auto bg-white p-3 rounded-md border border-gray-200">
                    <p class="text-center text-gray-500 italic">No past debates found.</p>
                </div>
            </div>
        </div>

        <!-- Right Panel: Debate Log and Input -->
        <div class="md:w-2/3 p-6 flex flex-col">
            <h2 class="text-xl font-semibold text-gray-800 mb-4">Live Debate</h2>

            <!-- Simulated Metrics Display -->
            <div class="grid grid-cols-1 sm:grid-cols-2 gap-3 mb-4 text-sm text-gray-700">
                <div class="bg-gray-100 p-2 rounded-md flex items-center gap-2">
                    <svg class="icon-svg"><use href="#mic"></use></svg>
                    <span>Transcription Accuracy: <span class="font-semibold">SarvamAI STT (en-IN)</span></span>
                </div>
                <div class="bg-gray-100 p-2 rounded-md flex items-center gap-2">
                    <svg class="icon-svg"><use href="#book-open"></use></svg>
                    <span>Case Prep Quality: <span class="font-semibold">Evaluated by Judge</span></span>
                </div>
            </div>

            <!-- Debate Log Display -->
            <div id="debate-log" class="flex-grow bg-gray-50 p-4 rounded-lg border border-gray-200 mb-4 overflow-y-auto max-h-[calc(100vh-350px)] md:max-h-[calc(100vh-250px)] flex flex-col gap-3 shadow-inner">
                <p class="text-center text-gray-500 italic">Enter a topic and click "Start Debate" to begin!</p>
            </div>

            <!-- User Input Area -->
            <div id="user-input-area" class="relative flex items-center gap-2 hidden">
                <button id="mic-button" class="p-3 rounded-full transition-colors duration-200 shadow-md bg-gray-200 text-gray-700 hover:bg-gray-300" title="Start Speaking">
                    <svg class="icon-svg"><use href="#mic"></use></svg>
                </button>
                <textarea id="user-text-input" placeholder="Type or speak your argument here..." class="flex-grow p-3 pr-12 border border-blue-300 rounded-lg focus:ring-2 focus:ring-blue-400 focus:border-transparent outline-none text-sm resize-y min-h-[60px]" rows="2"></textarea>
                <button id="send-button" class="absolute right-3 top-1/2 -translate-y-1/2 bg-blue-600 text-white p-2 rounded-full hover:bg-blue-700 transition-colors duration-200 shadow-md">
                    <svg class="icon-svg"><use href="#send"></use></svg>
                </button>
            </div>
            <div id="loading-indicator" class="flex justify-center items-center p-2 hidden">
                <svg class="icon-svg loader"><use href="#loader-2"></use></svg>
                <span class="ml-2 text-gray-600">AI is thinking...</span>
            </div>
            <div id="speaking-indicator" class="flex justify-center items-center p-2 text-blue-600 hidden">
                <svg class="icon-svg bot animate-pulse"><use href="#bot"></use></svg>
                <span class="ml-2">AI Speaking...</span>
            </div>
            <div id="feedback-indicator" class="text-center text-gray-600 italic hidden">Generating judge feedback...</div>
            <div id="judge-feedback-display" class="bg-green-50 p-4 rounded-lg border border-green-200 mt-4 shadow-md hidden">
                <h3 class="text-md font-semibold text-green-800 flex items-center gap-2 mb-2">
                    <svg class="icon-svg"><use href="#award"></use></svg> Judge's Feedback
                </h3>
                <p id="feedback-text" class="text-sm text-gray-700 whitespace-pre-wrap"></p>
            </div>
        </div>
    </div>

    <!-- Lucide Icons Definition (for use with <use href="#icon-name">) -->
    <svg style="display: none;">
        <defs>
            <symbol id="send" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><line x1="22" y1="2" x2="11" y2="13"></line><polygon points="22 2 15 22 11 13 2 9 22 2"></polygon></symbol>
            <symbol id="mic" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><path d="M12 2a3 3 0 0 0-3 3v7a3 3 0 0 0 6 0V5a3 3 0 0 0-3-3z"></path><path d="M19 10v2a7 7 0 0 1-14 0v-2"></path><line x1="12" y1="19" x2="12" y2="22"></line></symbol>
            <symbol id="refresh-cw" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><path d="M23 4v6h-6"></path><path d="M2.5 9.4c.7-5.5 5.1-9.4 10.5-9.4C18.5 0 23 4.5 23 10c0 4.5-3.4 8.2-7.9 8.9"></path><path d="M1 20v-6h6"></path><path d="M21.5 14.6c-.7 5.5-5.1 9.4-10.5 9.4C5.5 24 1 19.5 1 14c0-4.5 3.4-8.2 7.9-8.9"></path></symbol>
            <symbol id="message-square" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><path d="M21 15a2 2 0 0 1-2 2H7l-4 4V5a2 2 0 0 1 2-2h14a2 2 0 0 1 2 2z"></path></symbol>
            <symbol id="award" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><circle cx="12" cy="8" r="7"></circle><polyline points="8.21 13.89 7 22 12 18 17 22 15.79 13.89"></polyline></symbol>
            <symbol id="user" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><path d="M19 21v-2a4 4 0 0 0-4-4H9a4 4 0 0 0-4 4v2"></path><circle cx="12" cy="7" r="4"></circle></symbol>
            <symbol id="bot" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><path d="M12 8V4H8"></path><path d="M22 10a2 2 0 0 1-2 2H4a2 2 0 0 1-2-2V6a2 2 0 0 1 2-2h3.5L7 2h10l-.5 2H20a2 2 0 0 1 2 2z"></path><path d="M2 12v8a2 2 0 0 0 2 2h16a2 2 0 0 0 2-2v-8"></path><path d="M12 18h.01"></path><path d="M17 12h.01"></path><path d="M7 12h.01"></path></symbol>
            <symbol id="loader-2" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><path d="M12 2v4"></path><path d="M12 18v4"></path><path d="M4.93 4.93l2.83 2.83"></path><path d="M16.24 16.24l2.83 2.83"></path><path d="M2 12h4"></path><path d="M18 12h4"></path><path d="M4.93 19.07l2.83-2.83"></path><path d="M16.24 7.76l2.83-2.83"></path></symbol>
            <symbol id="book-open" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><path d="M2 3h6a4 4 0 0 1 4 4v14a3 3 0 0 0-3-3H2z"></path><path d="M22 3h-6a4 4 0 0 0-4 4v14a3 3 0 0 1 3-3h6z"></path></symbol>
            <symbol id="log-in" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><path d="M15 3h4a2 2 0 0 1 2 2v14a2 2 0 0 1-2 2h-4"></path><polyline points="10 17 15 12 10 7"></polyline><line x1="15" y1="12" x2="3" y2="12"></line></symbol>
            <symbol id="log-out" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><path d="M9 21H5a2 2 0 0 1-2-2V5a2 2 0 0 1 2-2h4"></path><polyline points="16 17 21 12 16 7"></polyline><line x1="21" y1="12" x2="9" y2="12"></line></line></symbol>
            <symbol id="user-plus" viewBox="0 0 24 24" fill="none" stroke="currentColor" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"><path d="M16 21v-2a4 4 0 0 0-4-4H6a4 4 0 0 0-4 4v2"></path><circle cx="9" cy="7" r="4"></circle><line x1="19" y1="8" x2="19" y2="14"></line><line x1="22" y1="11" x2="16" y2="11"></line></symbol>
        </defs>
    </svg>

    <script>
        // Global state variables (mimicking React state for this simplified example)
        let debateLog = [];
        let userInput = '';
        let isLoading = false;
        let debateState = 'idle'; // 'idle', 'user_turn', 'ai_turn', 'feedback', 'feedback_complete'
        let aiSkillLevel = 'intermediate';
        let judgeFeedback = '';
        let topic = '';
        let userCase = '';
        let currentTurn = 0;
        let userSide = 'pro'; // 'pro' or 'con'

        let isListening = false;
        let isSpeaking = false;
        let mediaRecorder = null; // MediaRecorder instance for audio capture
        let audioChunks = [];
        let audioPlayer = null; // Audio object for SarvamAI TTS playback

        let currentUser = null;
        let currentUserId = null;
        let authError = '';

        // DOM Elements
        const debateLogEl = document.getElementById('debate-log');
        const userInputEl = document.getElementById('user-text-input');
        const sendButton = document.getElementById('send-button');
        const micButton = document.getElementById('mic-button');
        const loadingIndicator = document.getElementById('loading-indicator');
        const speakingIndicator = document.getElementById('speaking-indicator');
        const feedbackIndicator = document.getElementById('feedback-indicator');
        const judgeFeedbackDisplay = document.getElementById('judge-feedback-display');
        const feedbackTextEl = document.getElementById('feedback-text');
        const topicInput = document.getElementById('topic');
        const skillLevelSelect = document.getElementById('skill-level');
        const userCaseTextarea = document.getElementById('user-case');
        const startDebateButton = document.getElementById('start-debate-button');
        const resetDebateButton = document.getElementById('reset-debate-button');
        const userInputArea = document.getElementById('user-input-area');
        const userSideRadios = document.querySelectorAll('input[name="userSide"]');
        const userDisplayEl = document.getElementById('user-display');
        const authButton = document.getElementById('auth-button');
        const authForms = document.getElementById('auth-forms');
        const emailInput = document.getElementById('email-input');
        const passwordInput = document.getElementById('password-input');
        const emailLoginButton = document.getElementById('email-login-button');
        const emailRegisterButton = document.getElementById('email-register-button');
        const passwordResetButton = document.getElementById('password-reset-button');
        const authErrorDisplay = document.getElementById('auth-error-display');
        const fullUserIdDisplay = document.getElementById('full-user-id-display');
        const userIdValueEl = document.getElementById('user-id-value');
        const pastDebatesSection = document.getElementById('past-debates-section');
        const pastDebatesList = document.getElementById('past-debates-list');

        // --- Utility Functions ---
        function updateUI() {
            // Update debate log
            debateLogEl.innerHTML = '';
            if (debateLog.length === 0 && debateState === 'idle') {
                debateLogEl.innerHTML = '<p class="text-center text-gray-500 italic">Enter a topic and click "Start Debate" to begin!</p>';
            } else {
                debateLog.forEach(entry => {
                    const entryDiv = document.createElement('div');
                    entryDiv.className = `debate-log-entry ${entry.speaker}`;
                    const iconSvg = `<svg class="icon-svg ${entry.speaker}"><use href="#${entry.speaker === 'user' ? 'user' : entry.speaker === 'ai' ? 'bot' : 'message-square'}"></use></svg>`;
                    entryDiv.innerHTML = `${iconSvg}<p class="text-sm text-gray-800 break-words flex-grow">${entry.text}</p>`;
                    debateLogEl.appendChild(entryDiv);
                });
            }
            debateLogEl.scrollTop = debateLogEl.scrollHeight; // Auto-scroll

            // Update input area and indicators
            userInputEl.value = userInput;
            userInputEl.placeholder = debateState === 'user_turn' ? (isListening ? "Listening..." : "Type or speak your argument here...") : "Waiting for AI response...";
            userInputEl.disabled = isLoading || debateState !== 'user_turn' || isListening;
            micButton.disabled = isLoading || debateState !== 'user_turn'; // Always enable mic if it's user's turn and not loading
            micButton.classList.toggle('bg-red-500', isListening);
            micButton.classList.toggle('text-white', isListening);
            micButton.classList.toggle('animate-pulse', isListening);
            micButton.classList.toggle('bg-gray-200', !isListening);
            micButton.classList.toggle('text-gray-700', !isListening);

            sendButton.disabled = isLoading || !userInput.trim() || debateState !== 'user_turn';

            loadingIndicator.classList.toggle('hidden', !isLoading);
            speakingIndicator.classList.toggle('hidden', !isSpeaking);
            feedbackIndicator.classList.toggle('hidden', debateState !== 'feedback');
            judgeFeedbackDisplay.classList.toggle('hidden', !(debateState === 'feedback_complete' && judgeFeedback));
            feedbackTextEl.textContent = judgeFeedback;

            startDebateButton.classList.toggle('hidden', debateState !== 'idle');
            resetDebateButton.classList.toggle('hidden', debateState === 'idle');
            userInputArea.classList.toggle('hidden', !(debateState === 'user_turn' || debateState === 'ai_turn'));

            // Control form elements based on debate state
            const isDebateActive = debateState !== 'idle';
            topicInput.disabled = isDebateActive;
            skillLevelSelect.disabled = isDebateActive;
            userCaseTextarea.disabled = isDebateActive;
            userSideRadios.forEach(radio => radio.disabled = isDebateActive);
            userCaseTextarea.placeholder = `Outline your main arguments for the ${userSide.toUpperCase()} side here. This helps the judge evaluate your preparation.`;

            // Auth UI
            if (currentUser) {
                userDisplayEl.innerHTML = `<svg class="icon-svg user"><use href="#user"></use></svg><span>${currentUser.displayName || currentUser.email || `User ID: ${currentUserId.substring(0, 8)}...`}</span>`;
                authButton.innerHTML = `<svg class="icon-svg"><use href="#log-out"></use></svg> Logout`;
                authButton.onclick = handleSignOut;
                authForms.classList.add('hidden');
                fullUserIdDisplay.classList.remove('hidden');
                userIdValueEl.textContent = currentUserId;
            } else {
                userDisplayEl.innerHTML = `<svg class="icon-svg user"><use href="#user"></use></svg><span>Not logged in</span>`;
                authButton.innerHTML = `<svg class="icon-svg"><use href="#log-in"></use></svg> Google Login`;
                authButton.onclick = handleGoogleSignIn;
                authForms.classList.remove('hidden');
                fullUserIdDisplay.classList.add('hidden');
            }
            authErrorDisplay.textContent = authError;
            authErrorDisplay.classList.toggle('hidden', !authError);

            // Past Debates Section
            if (currentUserId && pastDebates.length > 0) {
                pastDebatesSection.classList.remove('hidden');
                pastDebatesList.innerHTML = '';
                pastDebates.forEach(debate => {
                    const debateItem = document.createElement('div');
                    debateItem.className = 'p-2 border-b border-gray-100 last:border-b-0 cursor-pointer hover:bg-gray-50 rounded-md transition-colors duration-150';
                    debateItem.innerHTML = `
                        <p class="text-sm font-medium text-gray-800">${debate.topic}</p>
                        <p class="text-xs text-gray-500">
                            ${debate.timestamp ? new Date(debate.timestamp).toLocaleString() : 'N/A'} - AI: ${debate.aiSkillLevel} - Your side: ${debate.userSide?.toUpperCase() || 'N/A'}
                        </p>
                    `;
                    debateItem.onclick = () => loadPastDebate(debate);
                    pastDebatesList.appendChild(debateItem);
                });
            } else if (currentUserId) {
                pastDebatesSection.classList.remove('hidden');
                pastDebatesList.innerHTML = '<p class="text-center text-gray-500 italic">No past debates found.</p>';
            } else {
                pastDebatesSection.classList.add('hidden');
            }
        }

        // --- Firebase Client-Side Functions ---
        window.addEventListener('load', () => {
            if (window.auth && window.db) {
                window.onAuthStateChanged(window.auth, async (user) => {
                    if (user) {
                        currentUser = user;
                        currentUserId = user.uid;
                    } else {
                        try {
                            if (window.initialAuthToken) {
                                await window.signInWithCustomToken(window.auth, window.initialAuthToken);
                            } else {
                                await window.signInAnonymously(window.auth);
                            }
                        } catch (error) {
                            console.error("Error during anonymous sign-in or custom token sign-in:", error);
                            currentUserId = crypto.randomUUID(); // Fallback to a random ID
                        }
                    }
                    updateUI(); // Update UI after auth state is known
                    if (currentUserId) fetchPastDebates(); // Fetch debates once user ID is set
                });
            } else {
                console.error("Firebase SDKs not loaded or initialized correctly.");
                userDisplayEl.textContent = "Firebase Error";
                alert("Firebase SDKs failed to load. Please check your browser console.");
            }
        });

        async function fetchPastDebates() {
            if (!currentUserId || !window.db) return;
            try {
                const response = await fetch(`/api/debates/load/${currentUserId}`);
                if (!response.ok) throw new Error(`HTTP error! status: ${response.status}`);
                const data = await response.json();
                pastDebates = data;
                updateUI();
            } catch (error) {
                console.error("Error fetching past debates:", error);
                authError = `Failed to load past debates: ${error.message}`;
                updateUI();
            }
        }

        async function handleEmailLogin() {
            authError = '';
            try {
                await window.signInWithEmailAndPassword(window.auth, emailInput.value, passwordInput.value);
            } catch (error) {
                authError = error.message;
                console.error("Error signing in with email:", error);
            } finally {
                updateUI();
            }
        }

        async function handleEmailRegister() {
            authError = '';
            try {
                await window.createUserWithEmailAndPassword(window.auth, emailInput.value, passwordInput.value);
                alert('Registration successful! You are now logged in.');
            } catch (error) {
                authError = error.message;
                console.error("Error registering with email:", error);
            } finally {
                updateUI();
            }
        }

        async function handlePasswordReset() {
            authError = '';
            if (!emailInput.value) {
                authError = 'Please enter your email to reset password.';
                updateUI();
                return;
            }
            try {
                await window.sendPasswordResetEmail(window.auth, emailInput.value);
                alert('Password reset email sent! Check your inbox.');
            } catch (error) {
                authError = error.message;
                console.error("Error sending password reset email:", error);
            } finally {
                updateUI();
            }
        }

        async function handleGoogleSignIn() {
            authError = '';
            const provider = new window.GoogleAuthProvider();
            try {
                await window.signInWithPopup(window.auth, provider);
            } catch (error) {
                authError = error.message;
                console.error("Error signing in with Google:", error);
            } finally {
                updateUI();
            }
        }

        async function handleSignOut() {
            authError = '';
            try {
                await window.signOut(window.auth);
                debateLog = [];
                judgeFeedback = '';
                topic = '';
                userCase = '';
                currentTurn = 0;
                debateState = 'idle';
                userInput = '';
                emailInput.value = '';
                passwordInput.value = '';
            } catch (error) {
                authError = error.message;
                console.error("Error signing out:", error);
            } finally {
                updateUI();
            }
        }

        // --- Speech-to-Text (SarvamAI via Backend) ---
        async function startRecording() {
            if (isListening) return; // Already listening

            try {
                const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
                mediaRecorder = new MediaRecorder(stream);
                audioChunks = [];

                mediaRecorder.ondataavailable = (event) => {
                    audioChunks.push(event.data);
                };

                mediaRecorder.onstop = async () => {
                    isListening = false;
                    updateUI();
                    const audioBlob = new Blob(audioChunks, { type: 'audio/webm' }); // Use webm for broader compatibility
                    const reader = new FileReader();
                    reader.readAsDataURL(audioBlob);
                    reader.onloadend = async () => {
                        const base64Audio = reader.result.split(',')[1]; // Get base64 part
                        isLoading = true; // Show loading for transcription
                        updateUI();
                        try {
                            const response = await fetch('/api/speech_to_text', {
                                method: 'POST',
                                headers: { 'Content-Type': 'application/json' },
                                body: JSON.stringify({ audioBase64: base64Audio })
                            });
                            const data = await response.json();
                            if (data.transcript) {
                                userInput = data.transcript;
                                updateUI();
                                // Automatically submit the transcribed text
                                if (debateState === 'user_turn') {
                                    handleUserSubmit(userInput);
                                }
                            } else {
                                alert('Failed to transcribe audio: ' + (data.error || 'Unknown error'));
                            }
                        } catch (error) {
                            console.error("Error sending audio for transcription:", error);
                            alert('Error transcribing audio. Please try again.');
                        } finally {
                            isLoading = false;
                            updateUI();
                        }
                    };
                    // Stop microphone stream tracks
                    stream.getTracks().forEach(track => track.stop());
                };

                mediaRecorder.start();
                isListening = true;
                userInput = ''; // Clear input while recording
                updateUI();
            } catch (error) {
                console.error("Error accessing microphone:", error);
                alert('Microphone access denied or failed: ' + error.message);
                isListening = false;
                updateUI();
            }
        }

        function stopRecording() {
            if (mediaRecorder && mediaRecorder.state === 'recording') {
                mediaRecorder.stop();
            }
        }

        function toggleSpeechRecognition() {
            if (isListening) {
                stopRecording();
            } else {
                startRecording();
            }
        }

        // --- Text-to-Speech (SarvamAI via Backend, Fallback to Browser Native) ---
        async function speakText(text) {
            isSpeaking = true;
            updateUI();

            // Stop any ongoing browser speech
            if (window.speechSynthesis.speaking) {
                window.speechSynthesis.cancel();
            }
            // Stop any ongoing SarvamAI audio
            if (audioPlayer) {
                audioPlayer.pause();
                audioPlayer.src = '';
            }

            try {
                // Call backend for TTS, backend will call SarvamAI
                const response = await fetch('/api/debate/ai_turn', { // This endpoint also handles TTS now
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    // This payload is simplified for TTS only, the actual ai_turn route expects more
                    // For a clean separation, you might want a dedicated /api/text_to_speech endpoint
                    body: JSON.stringify({ textToSpeak: text })
                });
                const data = await response.json();

                if (data.audioBase64) {
                    const audioUrl = `data:audio/mpeg;base64,${data.audioBase64}`; // Assuming MPEG/MP3
                    if (!audioPlayer) {
                        audioPlayer = new Audio();
                    }
                    audioPlayer.src = audioUrl;
                    audioPlayer.play();

                    audioPlayer.onended = () => {
                        isSpeaking = false;
                        updateUI();
                        URL.revokeObjectURL(audioUrl);
                    };
                    audioPlayer.onerror = (e) => {
                        console.error("Audio playback error:", e);
                        isSpeaking = false;
                        updateUI();
                        URL.revokeObjectURL(audioUrl);
                        // Fallback to browser TTS if SarvamAI playback fails
                        console.warn("SarvamAI audio playback failed. Falling back to browser's native Text-to-Speech.");
                        const utterance = new SpeechSynthesisUtterance(text);
                        utterance.lang = 'en-IN';
                        window.speechSynthesis.speak(utterance);
                        utterance.onend = () => { isSpeaking = false; updateUI(); };
                    };
                } else {
                    throw new Error("No audio data received from SarvamAI TTS via backend.");
                }

            } catch (error) {
                console.error("Error with SarvamAI speech via backend:", error);
                isSpeaking = false;
                updateUI();
                // Fallback to browser TTS if SarvamAI fails
                console.warn("SarvamAI TTS via backend failed. Falling back to browser's native Text-to-Speech.");
                const utterance = new SpeechSynthesisUtterance(text);
                utterance.lang = 'en-IN';
                window.speechSynthesis.speak(utterance);
                utterance.onend = () => { isSpeaking = false; updateUI(); };
            }
        }

        // --- Debate Logic Functions ---
        function getDebateTurnsConfig(chosenSide) {
            const aiSkill = skillLevelSelect.value; // Get current AI skill level
            const proPrompt = (topic) => `You are the user. The debate topic is "${topic}". Please make your opening statement for the PRO side. Be concise and persuasive.`;
            const conPrompt = (topic) => `You are the user. The debate topic is "${topic}". Please make your opening statement for the CON side. Be concise and persuasive.`;

            const aiProRebuttal = (topic, lastUserStatement) => `You are an AI debater. The debate topic is "${topic}". Your opponent (the user) just made their opening statement: "${lastUserStatement}". Please provide a concise rebuttal against their points, arguing for the PRO side. Your skill level is "${aiSkill}".`;
            const aiConRebuttal = (topic, lastUserStatement) => `You are an AI debater. The debate topic is "${topic}". Your opponent (the user) just made their opening statement: "${lastUserStatement}". Please provide a concise rebuttal against their points, arguing for the CON side. Your skill level is "${aiSkill}".`;

            const aiProClosing = (topic, lastUserStatement) => `You are an AI debater. The debate topic is "${topic}". Your opponent (the user) just made their rebuttal: "${lastUserStatement}". Please provide a concise closing statement for the PRO side, summarizing your main points and refuting the opponent's. Your skill level is "${aiSkill}".`;
            const aiConClosing = (topic, lastUserStatement) => `You are an AI debater. The debate topic is "${topic}". Your opponent (the user) just made their rebuttal: "${lastUserStatement}". Please provide a concise closing statement for the CON side, summarizing your main points and refuting the opponent's. Your skill level is "${aiSkill}".`;

            if (chosenSide === 'pro') {
                return [
                    { speaker: 'user', type: 'opening', prompt: proPrompt },
                    { speaker: 'ai', type: 'rebuttal', prompt: aiConRebuttal },
                    { speaker: 'user', type: 'rebuttal', prompt: (topic, lastAiStatement) => `You are the user. The debate topic is "${topic}". The AI just made a rebuttal: "${lastAiStatement}". Please provide your rebuttal to the AI's points, reinforcing your PRO side.` },
                    { speaker: 'ai', type: 'closing', prompt: aiConClosing },
                    { speaker: 'user', type: 'closing', prompt: (topic, lastAiStatement) => `You are the user. The debate topic is "${topic}". The AI just made its closing statement: "${lastAiStatement}". Please provide your closing statement for the PRO side.` },
                ];
            } else { // chosenSide === 'con'
                return [
                    { speaker: 'user', type: 'opening', prompt: conPrompt },
                    { speaker: 'ai', type: 'rebuttal', prompt: aiProRebuttal },
                    { speaker: 'user', type: 'rebuttal', prompt: (topic, lastAiStatement) => `You are the user. The debate topic is "${topic}". The AI just made a rebuttal: "${lastAiStatement}". Please provide your rebuttal to the AI's points, reinforcing your CON side.` },
                    { speaker: 'ai', type: 'closing', prompt: aiProClosing },
                    { speaker: 'user', type: 'closing', prompt: (topic, lastAiStatement) => `You are the user. The debate topic is "${topic}". The AI just made its closing statement: "${lastAiStatement}". Please provide your closing statement for the CON side.` },
                ];
            }
        }

        async function startDebate() {
            if (!topicInput.value.trim()) {
                alert('Please enter a debate topic to start!');
                return;
            }
            debateLog = [];
            judgeFeedback = '';
            currentTurn = 0;
            debateState = 'user_turn';
            topic = topicInput.value.trim();
            aiSkillLevel = skillLevelSelect.value;
            userSide = document.querySelector('input[name="userSide"]:checked').value;
            userCase = userCaseTextarea.value.trim();

            debateLog.push({ speaker: 'system', text: `Debate started on topic: "${topic}". You are arguing the ${userSide.toUpperCase()} side. AI is arguing the ${userSide === 'pro' ? 'CON' : 'PRO'} side.` });
            updateUI();
        }

        async function handleUserSubmit(textToSubmit) {
            if (!textToSubmit.trim() || isLoading) return;

            const userStatement = textToSubmit.trim();
            debateLog.push({ speaker: 'user', text: userStatement });
            userInput = '';
            updateUI();

            const debateTurnsConfig = getDebateTurnsConfig(userSide);
            const nextTurnIndex = currentTurn + 1;

            if (nextTurnIndex < debateTurnsConfig.length) {
                currentTurn = nextTurnIndex;
                debateState = 'ai_turn';
                updateUI();
                await generateAiResponse(nextTurnIndex, userStatement);
            } else {
                debateState = 'feedback';
                updateUI();
                await getJudgeFeedback();
            }
        }

        async function generateAiResponse(turnIndex, lastOpponentStatement) {
            isLoading = true;
            updateUI();
            try {
                const debateTurnsConfig = getDebateTurnsConfig(userSide);
                const currentTurnData = debateTurnsConfig[turnIndex];
                const previousAiArguments = debateLog
                    .filter(log => log.speaker === 'ai' && log.type !== 'system')
                    .map(log => log.text)
                    .join('\n');

                const prompt = currentTurnData.prompt(topic, lastOpponentStatement, previousAiArguments);

                // Call backend for AI text and TTS audio
                const response = await fetch('/api/debate/ai_turn', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({
                        topic: topic,
                        lastOpponentStatement: lastOpponentStatement,
                        currentTurnIndex: turnIndex,
                        aiSkillLevel: aiSkillLevel,
                        userSide: userSide,
                        debateLog: debateLog
                    })
                });
                const data = await response.json();

                let aiText = data.aiText || "I'm sorry, I couldn't generate a response.";
                debateLog.push({ speaker: 'ai', text: aiText });
                updateUI();

                if (data.audioBase64) {
                    await speakTextFromBase64(data.audioBase64);
                } else {
                    console.warn("Backend did not provide SarvamAI audio. Falling back to browser TTS.");
                    const utterance = new SpeechSynthesisUtterance(aiText);
                    utterance.lang = 'en-IN';
                    window.speechSynthesis.speak(utterance);
                    await new Promise(resolve => utterance.onend = resolve);
                }

                const nextTurnAfterAi = turnIndex + 1;
                if (nextTurnAfterAi < debateTurnsConfig.length) {
                    debateState = 'user_turn';
                } else {
                    debateState = 'feedback';
                }

            } catch (error) {
                console.error("Error generating AI response:", error);
                debateLog.push({ speaker: 'system', text: 'Error: Could not get AI response. Please try again.' });
                debateState = 'idle';
            } finally {
                isLoading = false;
                updateUI();
            }
        }

        async function getJudgeFeedback() {
            isLoading = true;
            updateUI();
            try {
                const response = await fetch('/api/debate/feedback', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({
                        topic: topic,
                        userCase: userCase,
                        debateLog: debateLog,
                        userSide: userSide
                    })
                });
                const data = await response.json();

                judgeFeedback = data.judgeFeedback || "Could not generate judge feedback.";
                debateState = 'feedback_complete';
                updateUI();

                if (data.audioBase64) {
                    await speakTextFromBase64(data.audioBase64);
                } else {
                    console.warn("Backend did not provide SarvamAI audio for feedback. Falling back to browser TTS.");
                    const utterance = new SpeechSynthesisUtterance(judgeFeedback);
                    utterance.lang = 'en-IN';
                    window.speechSynthesis.speak(utterance);
                    await new Promise(resolve => utterance.onend = resolve);
                }

                // Save the debate to Firestore via backend
                if (currentUserId) {
                    try {
                        const saveResponse = await fetch('/api/debates/save', {
                            method: 'POST',
                            headers: { 'Content-Type': 'application/json' },
                            body: JSON.stringify({
                                userId: currentUserId,
                                debateData: {
                                    topic: topic,
                                    userCase: userCase,
                                    debateLog: debateLog,
                                    judgeFeedback: judgeFeedback,
                                    aiSkillLevel: aiSkillLevel,
                                    userSide: userSide,
                                    // timestamp is added by backend
                                }
                            })
                        });
                        if (!saveResponse.ok) throw new Error(`Failed to save debate: ${saveResponse.statusText}`);
                        console.log("Debate saved successfully!");
                        fetchPastDebates(); // Refresh past debates list
                    } catch (saveError) {
                        console.error("Error saving debate to Firestore:", saveError);
                    }
                }

            } catch (error) {
                console.error("Error getting judge feedback:", error);
                judgeFeedback = 'Error: Could not get judge feedback.';
                updateUI();
            } finally {
                isLoading = false;
                updateUI();
            }
        }

        async function speakTextFromBase64(base64Audio) {
            isSpeaking = true;
            updateUI();
            if (audioPlayer) {
                audioPlayer.pause();
                audioPlayer.src = '';
            }
            if (!audioPlayer) {
                audioPlayer = new Audio();
            }
            const audioUrl = `data:audio/mpeg;base64,${base64Audio}`;
            audioPlayer.src = audioUrl;
            audioPlayer.play();

            return new Promise((resolve, reject) => {
                audioPlayer.onended = () => {
                    isSpeaking = false;
                    updateUI();
                    URL.revokeObjectURL(audioUrl);
                    resolve();
                };
                audioPlayer.onerror = (e) => {
                    console.error("Audio playback error:", e);
                    isSpeaking = false;
                    updateUI();
                    URL.revokeObjectURL(audioUrl);
                    reject(e);
                };
            });
        }


        function loadPastDebate(debate) {
            topicInput.value = debate.topic;
            userCaseTextarea.value = debate.userCase;
            debateLog = debate.debateLog;
            judgeFeedback = debate.judgeFeedback;
            skillLevelSelect.value = debate.aiSkillLevel;
            document.querySelector(`input[name="userSide"][value="${debate.userSide || 'pro'}"]`).checked = true;
            userSide = debate.userSide || 'pro';
            currentTurn = debate.debateLog.filter(entry => entry.speaker !== 'system').length;
            debateState = 'feedback_complete';
            updateUI();
        }

        // --- Event Listeners ---
        document.addEventListener('DOMContentLoaded', () => {
            // No longer calling initSpeechRecognition here.
            // MediaRecorder setup is now within startRecording function.

            userInputEl.addEventListener('input', (e) => { userInput = e.target.value; updateUI(); });
            userInputEl.addEventListener('keypress', (e) => {
                if (e.key === 'Enter' && !e.shiftKey) {
                    e.preventDefault();
                    handleUserSubmit(userInput);
                }
            });
            sendButton.addEventListener('click', () => handleUserSubmit(userInput));
            micButton.addEventListener('click', toggleSpeechRecognition);
            startDebateButton.addEventListener('click', startDebate);
            resetDebateButton.addEventListener('click', () => {
                if (audioPlayer) { audioPlayer.pause(); audioPlayer.src = ''; }
                if (window.speechSynthesis.speaking) { window.speechSynthesis.cancel(); }
                if (mediaRecorder && mediaRecorder.state === 'recording') { mediaRecorder.stop(); } // Stop recording if active

                isListening = false;
                isSpeaking = false;
                debateState = 'idle';
                topic = '';
                userCase = '';
                debateLog = [];
                judgeFeedback = '';
                currentTurn = 0;
                userInput = '';
                emailInput.value = '';
                passwordInput.value = '';
                topicInput.value = '';
                userCaseTextarea.value = '';
                skillLevelSelect.value = 'intermediate';
                document.querySelector('input[name="userSide"][value="pro"]').checked = true;
                userSide = 'pro';
                updateUI();
            });

            emailLoginButton.addEventListener('click', handleEmailLogin);
            emailRegisterButton.addEventListener('click', handleEmailRegister);
            passwordResetButton.addEventListener('click', handlePasswordReset);
            authButton.addEventListener('click', handleGoogleSignIn); // Initial state is Google Login
        });
    </script>
</body>
</html>

SyntaxError: invalid decimal literal (1976166886.py, line 18)